In [ ]:
# | default_exp airflow.aws_batch_executor

In [ ]:
# | export

import shlex
import tempfile
from pathlib import Path
from typing import *

import yaml
from airt.executor.subcommand import ClassCLICommand, CLICommandBase
from airt.helpers import slugify
from airt.logger import get_logger
from airt.patching import patch
from fastcore.script import Param, call_parse

from airt_service.airflow.base_executor import BaseAirflowExecutor, dag_template
from airt_service.airflow.utils import trigger_dag, wait_for_run_to_complete
from airt_service.aws.batch_utils import (
    _create_default_batch_environment_config,
    create_testing_batch_environment_ctx,
)
from airt_service.aws.utils import get_batch_environment_arns, get_queue_definition_arns
from airt_service.batch_job import get_environment_vars_for_batch_job
from airt_service.helpers import generate_random_string
from airt_service.sanitizer import sanitized_print

In [ ]:
import os
from datetime import timedelta
from time import sleep

import pytest
from airt.executor.subcommand import SimpleCLICommand
from airt.testing import activate_by_import

from airt_service.airflow.utils import list_dag_runs
from airt_service.db.models import create_user_for_testing

In [ ]:
test_username = create_user_for_testing(subscription_type="small")
display(test_username)

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
logger.info("Module loaded.")

In [ ]:
# | exporti


def setup_test_paths(td: str) -> Tuple[str, str]:
    d = Path(td)
    paths = [d / sd for sd in ["data", "model"]]
    sanitized_print(f"{paths=}")

    # create tmp dirs for data and model
    for p in paths:
        p.mkdir(parents=True, exist_ok=True)

    # RemotePaths: data_path is "read-only", while model_path can be used for both reading and writing between calls
    return tuple(f"local:{p}" for p in paths)  # type: ignore

In [ ]:
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)

data_path_url, model_path_url

In [ ]:
# | export

DEFAULT_EXEC_ENVIRONMENT = "preprocessing"

In [ ]:
# | export


class AirflowAWSBatchExecutor(BaseAirflowExecutor):
    def __init__(
        self,
        steps: List[CLICommandBase],
        region: str,
        exec_environments: Optional[List[Optional[str]]] = None,
        batch_environment_arn_path: Optional[Union[str, Path]] = None,
    ):
        """Constructs a new AirflowAWSBatchExecutor instance

        Args:
            steps: List of instances of either ClassCLICommand or SimpleCLICommand
            region: Region to execute
            exec_environments: List of execution environments to execute steps
        """
        self.region = region
        self.batch_environment_arn_path = batch_environment_arn_path

        if exec_environments is None:
            exec_environments = [DEFAULT_EXEC_ENVIRONMENT] * len(steps)

        if len(exec_environments) != len(steps):
            raise ValueError(
                f"len(exec_environments)={len(exec_environments)} != len(steps){len(steps)}"
            )

        existing_exec_environments = list(
            get_batch_environment_arns(
                self.region, self.batch_environment_arn_path
            ).keys()
        )

        self.exec_environments = []
        for exec_env in exec_environments:
            if exec_env is None:
                self.exec_environments.append(DEFAULT_EXEC_ENVIRONMENT)
                continue
            if exec_env not in existing_exec_environments:
                raise ValueError(
                    f"Invalid value {exec_env} given for exec environment; Allowed values are {existing_exec_environments}"
                )
            self.exec_environments.append(exec_env)

        self.exec_environments = [
            exec_env if exec_env is not None else DEFAULT_EXEC_ENVIRONMENT
            for exec_env in exec_environments
        ]

        super(AirflowAWSBatchExecutor, self).__init__(steps)

    def execute(
        self,
        *,
        description: str,
        tags: Union[str, List[str]],
        on_step_start: Optional[CLICommandBase] = None,
        on_step_end: Optional[CLICommandBase] = None,
        **kwargs: Any,
    ) -> Tuple[Path, str]:
        """Create DAG and execute steps in airflow

        Args:
            description: description of DAG
            tags: tags for DAG
            on_step_start: CLI to call before executing step/task in DAG
            on_step_end: CLI to call after executing step/task in DAG
            kwargs: keyword arguments needed for steps/tasks
        Returns:
            A tuple which contains dag file path and run id
        """
        raise NotImplementedError("Need to implement")

In [ ]:
def save_test_batch_environment_arns(folder: Path):
    test_batch_environment_arns = {
        "eu-west-1": {
            task: {
                arn: "arn:aws:batch:placeholder"
                for arn in [
                    "compute_environment_arn",
                    "job_definition_arn",
                    "job_queue_arn",
                ]
            }
            for task in ["csv_processing", "predictions", "preprocessing", "training"]
        }
    }

    folder = Path(folder)
    test_batch_environment_arn_path = folder / "batch_environment.yml"
    with open(test_batch_environment_arn_path, "w") as f:
        yaml.dump(test_batch_environment_arns, f, default_flow_style=False)

    return test_batch_environment_arn_path

In [ ]:
steps = [
    ClassCLICommand(
        executor_name="test-executor", class_name="MyTestExecutor", f_name="f"
    ),
    ClassCLICommand(
        executor_name="test-executor", class_name="MyTestExecutor", f_name="g"
    ),
]

In [ ]:
region = "eu-west-1"
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)

    test_batch_environment_arn_path = save_test_batch_environment_arns(d)
    abe = AirflowAWSBatchExecutor(
        steps=steps,
        region=region,
        batch_environment_arn_path=test_batch_environment_arn_path,
    )
    display(abe.exec_environments)
    assert abe.exec_environments == ["preprocessing"] * len(steps)

    with pytest.raises(ValueError) as e:
        abe = AirflowAWSBatchExecutor(
            steps=steps,
            region=region,
            exec_environments=["preprocessing"],
            batch_environment_arn_path=test_batch_environment_arn_path,
        )
    display(e)

    with pytest.raises(ValueError) as e:
        abe = AirflowAWSBatchExecutor(
            steps=steps,
            region=region,
            exec_environments=["gibberish", "gibberish"],
            batch_environment_arn_path=test_batch_environment_arn_path,
        )
    display(e)

In [ ]:
# | export


@patch  # type: ignore
def _create_step_template(
    self: AirflowAWSBatchExecutor,
    step: CLICommandBase,
    exec_environment: str,
    **kwargs: Any,
) -> str:
    """
    Create template for step

    Args:
        step: step to create template
        kwargs: keyword arguments for step
    Returns:
        Template for step
    """
    cli_command = step.to_cli(**kwargs)
    task_id = slugify(cli_command)

    batch_environment_vars = [
        dict(name=name, value=value)
        for name, value in get_environment_vars_for_batch_job().items()
    ]
    overrides = (
        dict(
            command=shlex.split(cli_command),
            environment=batch_environment_vars,
        )
        .__repr__()
        .replace("{", "{{")
        .replace("}", "}}")
    )

    job_queue_arn, job_definition_arn = get_queue_definition_arns(
        task=exec_environment,
        region=self.region,
        batch_environment_arn_path=self.batch_environment_arn_path,
    )

    task = f"""BatchOperator(task_id='{task_id}', job_definition="{job_definition_arn}", job_queue="{job_queue_arn}", job_name="{task_id}", overrides={overrides})"""

    return task

In [ ]:
region = "eu-west-1"
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)

    test_batch_environment_arn_path = save_test_batch_environment_arns(d)
    abe = AirflowAWSBatchExecutor(
        steps=steps,
        region=region,
        batch_environment_arn_path=test_batch_environment_arn_path,
    )
    actual = abe._create_step_template(
        steps[0],
        exec_environment="training",
        data_path_url=data_path_url,
        model_path_url=model_path_url,
    )
    display(actual)

In [ ]:
# | export


@patch  # type: ignore
def _create_dag_template(
    self: AirflowAWSBatchExecutor,
    on_step_start: Optional[CLICommandBase] = None,
    on_step_end: Optional[CLICommandBase] = None,
    **kwargs: Any,
) -> str:
    """
    Create DAG template with steps as tasks

    Args:
        on_step_start: CLI to call before executing step/task in DAG
        on_step_end: CLI to call after executing step/task in DAG
        kwargs: keyword arguments to pass to steps' CLI
    Returns:
        Generated DAG with steps as tasks
    """
    curr_dag_template = dag_template

    downstream_tasks = ""
    newline = "\n"
    tab = " " * 4

    existing_tasks = 0
    for i, step in enumerate(self.steps):
        if on_step_start is not None:
            curr_dag_template += f"""{newline}{tab}t{existing_tasks+1} = {self._create_step_template(on_step_start, self.exec_environments[i], step_count=i+1, **kwargs)}"""  # type: ignore
            existing_tasks += 1

        curr_dag_template += f"""{newline}{tab}t{existing_tasks+1} = {self._create_step_template(step, self.exec_environments[i], **kwargs)}"""  # type: ignore
        existing_tasks += 1

        if on_step_end is not None:
            curr_dag_template += f"""{newline}{tab}t{existing_tasks+1} = {self._create_step_template(on_step_end, self.exec_environments[i], step_count=i+1, **kwargs)}"""  # type: ignore
            existing_tasks += 1

    downstream_tasks = f"{newline}{tab}" + " >> ".join(
        [f"t{i}" for i in range(1, existing_tasks + 1)]
    )
    curr_dag_template += downstream_tasks

    return curr_dag_template

In [ ]:
region = "eu-west-1"
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)
    test_batch_environment_arn_path = save_test_batch_environment_arns(d)

    kwargs = {"data_path_url": data_path_url, "model_path_url": model_path_url}

    abe = AirflowAWSBatchExecutor(
        steps=steps,
        region=region,
        batch_environment_arn_path=test_batch_environment_arn_path,
    )

    on_step_start = SimpleCLICommand(command="sleep {step_count}")
    on_step_end = SimpleCLICommand(command="echo step {step_count} completed")
    sanitized_print(
        abe._create_dag_template(
            on_step_start=on_step_start, on_step_end=on_step_end, **kwargs
        )
    )

In [ ]:
# | eval: false
# Test case for AirflowAWSBatchExecutor._create_dag
region = "eu-west-1"
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)
    steps = [
        ClassCLICommand(
            executor_name="test-executor", class_name="MyTestExecutor", f_name="f"
        ),
        ClassCLICommand(
            executor_name="test-executor", class_name="MyTestExecutor", f_name="g"
        ),
    ]
    exec_environments = ["training", None]
    on_step_start = SimpleCLICommand(command="sleep {step_count}")
    on_step_end = SimpleCLICommand(command="echo step {step_count} completed")

    td = Path(d)
    env_config_path = td / "env_config.yaml"
    created_env_info_path = td / "output_file.yaml"
    _create_default_batch_environment_config(
        prefix=f"airflow_batch_create_dag_testing_{generate_random_string()}",
        output_path=env_config_path,
        regions=[region],
    )

    with open(env_config_path) as f:
        env_config = yaml.safe_load(f)
    display(f"{env_config=}")
    with create_testing_batch_environment_ctx(
        input_yaml_path=env_config_path, output_yaml_path=created_env_info_path
    ):
        abe = AirflowAWSBatchExecutor(
            steps=steps,
            region=region,
            exec_environments=exec_environments,
            batch_environment_arn_path=created_env_info_path,
        )
        dag_id, dag_file_path = abe._create_dag(
            data_path_url=data_path_url,
            model_path_url=model_path_url,
            #         schedule_interval="@weekly",
            schedule_interval=None,
            description="test description",
            tags="test_tag",
            on_step_start=on_step_start,
            on_step_end=on_step_end,
        )

        display(f"{dag_file_path=}")
        dag_id = str(dag_file_path).split("/")[-1].split(".py")[0]

        sleep(15)

        dag_runs = list_dag_runs(dag_id=dag_id)
        display(f"{dag_runs=}")

        run_id = trigger_dag(dag_id=dag_id, conf={})

        #     run_id = dag_runs[0]["run_id"]
        display(run_id)
        state = wait_for_run_to_complete(dag_id=dag_id, run_id=run_id, timeout=3600)
        display(state)
        dag_file_path.unlink()

In [ ]:
# | eval: false
# Test case for AirflowAWSBatchExecutor.schedule
region = "eu-west-1"
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)
    steps = [
        ClassCLICommand(
            executor_name="test-executor", class_name="MyTestExecutor", f_name="f"
        ),
        ClassCLICommand(
            executor_name="test-executor", class_name="MyTestExecutor", f_name="g"
        ),
    ]
    exec_environments = ["csv_processing", "preprocessing"]
    on_step_start = SimpleCLICommand(command="sleep {step_count}")
    on_step_end = SimpleCLICommand(command="echo step {step_count} completed")

    td = Path(d)
    env_config_path = td / "env_config.yaml"
    created_env_info_path = td / "output_file.yaml"
    _create_default_batch_environment_config(
        prefix=f"airflow_batch_schedule_testing_{generate_random_string()}",
        output_path=env_config_path,
        regions=[region],
    )

    with open(env_config_path) as f:
        env_config = yaml.safe_load(f)
    display(f"{env_config=}")
    with create_testing_batch_environment_ctx(
        input_yaml_path=env_config_path, output_yaml_path=created_env_info_path
    ):
        abe = AirflowAWSBatchExecutor(
            steps=steps,
            region=region,
            exec_environments=exec_environments,
            batch_environment_arn_path=created_env_info_path,
        )
        dag_file_path = abe.schedule(
            data_path_url=data_path_url,
            model_path_url=model_path_url,
            #         schedule_interval="@weekly",
            schedule_interval=timedelta(days=7),
            description="test description",
            tags="test_tag",
            on_step_start=on_step_start,
            on_step_end=on_step_end,
        )

        display(f"{dag_file_path=}")
        dag_id = str(dag_file_path).split("/")[-1].split(".py")[0]

        sleep(15)

        dag_runs = list_dag_runs(dag_id=dag_id)
        display(f"{dag_runs=}")

        run_id = trigger_dag(dag_id=dag_id, conf={})

        #     run_id = dag_runs[0]["run_id"]
        display(run_id)
        state = wait_for_run_to_complete(dag_id=dag_id, run_id=run_id, timeout=3600)
        display(state)
        dag_file_path.unlink()

In [ ]:
# | export


@patch  # type: ignore
def execute(
    self: AirflowAWSBatchExecutor,
    *,
    description: str,
    tags: Union[str, List[str]],
    on_step_start: Optional[CLICommandBase] = None,
    on_step_end: Optional[CLICommandBase] = None,
    **kwargs: Any,
) -> Tuple[Path, str]:
    """Create DAG and execute steps in airflow

    Args:
        description: description of DAG
        tags: tags for DAG
        on_step_start: CLI to call before executing step/task in DAG
        on_step_end: CLI to call after executing step/task in DAG
        kwargs: keyword arguments needed for steps/tasks
    Returns:
        A tuple which contains dag file path and run id
    """
    schedule_interval = None
    dag_id, dag_file_path = self._create_dag(
        schedule_interval=schedule_interval,
        description=description,
        tags=tags,
        on_step_start=on_step_start,
        on_step_end=on_step_end,
        **kwargs,
    )

    run_id = trigger_dag(dag_id=dag_id, conf={})
    return dag_file_path, run_id

In [ ]:
# | eval: false

region = "eu-west-1"
with tempfile.TemporaryDirectory() as d:
    data_path_url, model_path_url = setup_test_paths(d)

    steps = [
        ClassCLICommand(
            executor_name="test-executor", class_name="MyTestExecutor", f_name="f"
        ),
        ClassCLICommand(
            executor_name="test-executor", class_name="MyTestExecutor", f_name="g"
        ),
    ]
    exec_environments = ["training", "predictions"]
    on_step_start = SimpleCLICommand(command="sleep {step_count}")
    on_step_end = SimpleCLICommand(command="echo step {step_count} completed")

    td = Path(d)
    env_config_path = td / "env_config.yaml"
    created_env_info_path = td / "output_file.yaml"
    _create_default_batch_environment_config(
        prefix=f"airflow_batch_execute_testing_{generate_random_string()}",
        output_path=env_config_path,
        regions=[region],
    )

    with open(env_config_path) as f:
        env_config = yaml.safe_load(f)
    display(f"{env_config=}")
    with create_testing_batch_environment_ctx(
        input_yaml_path=env_config_path, output_yaml_path=created_env_info_path
    ):
        abe = AirflowAWSBatchExecutor(
            steps=steps,
            region=region,
            exec_environments=exec_environments,
            batch_environment_arn_path=created_env_info_path,
        )

        dag_file_path, run_id = abe.execute(
            description="test description",
            tags="test_tag",
            on_step_start=on_step_start,
            on_step_end=on_step_end,
            data_path_url=data_path_url,
            model_path_url=model_path_url,
        )
        display(dag_file_path)
        display(run_id)

        dag_id = str(dag_file_path).split("/")[-1].split(".py")[0]
        state = wait_for_run_to_complete(dag_id=dag_id, run_id=run_id, timeout=3600)
        display(state)
        dag_file_path.unlink()

In [ ]:
# | export


def _test_aws_batch_executor(region: str = "eu-west-1") -> None:
    with tempfile.TemporaryDirectory() as d:
        data_path_url, model_path_url = setup_test_paths(d)

        steps = [
            ClassCLICommand(
                executor_name="test-executor", class_name="MyTestExecutor", f_name="f"
            )
        ]
        exec_environments = ["training"]

        td = Path(d)
        env_config_path = td / "env_config.yaml"
        created_env_info_path = td / "output_file.yaml"

        prefix = f"airflow_batch_execute_testing_{generate_random_string()}"
        regions = [region]
        _create_default_batch_environment_config(
            prefix=prefix,
            output_path=env_config_path,
            regions=regions,
        )

        with open(env_config_path) as f:
            env_config = yaml.safe_load(f)
        logger.info(f"{env_config=}")
        with create_testing_batch_environment_ctx(
            input_yaml_path=env_config_path, output_yaml_path=created_env_info_path  # type: ignore
        ):
            abe = AirflowAWSBatchExecutor(
                steps=steps,
                region=region,
                exec_environments=exec_environments,  # type: ignore
                batch_environment_arn_path=created_env_info_path,
            )

            dag_file_path, run_id = abe.execute(
                description="test description",
                tags="test_tag",
                data_path_url=data_path_url,
                model_path_url=model_path_url,
            )
            logger.info(f"{dag_file_path=}")
            logger.info(f"{run_id=}")

            dag_id = str(dag_file_path).split("/")[-1].split(".py")[0]
            state = wait_for_run_to_complete(dag_id=dag_id, run_id=run_id, timeout=1200)
            logger.info(f"{state=}")
            dag_file_path.unlink()

In [ ]:
# | export


@call_parse  # type: ignore
def test_aws_batch_executor(region: Param("region", str) = "eu-west-1"):  # type: ignore
    """
    Create throw away environment for aws batch and execute airflow batch executor
    """
    _test_aws_batch_executor(region=region)

In [ ]:
# | eval: false
test_aws_batch_executor()